# Starting with quality control

In [1]:
import os

os.chdir("/media/dftortosa/Windows/Users/dftor/Documents/diego_docs/science/other_projects/australian_army_bishop/quality_control")

os.getcwd()

'/media/dftortosa/Windows/Users/dftor/Documents/diego_docs/science/other_projects/australian_army_bishop/quality_control'

## Data available

I have received a Illumina report with 3 files ([link](https://www.biostars.org/p/51928/)):

- The "FinalReport.txt" for Illumina raw genotype data generated from Genome Bead Studio for 2.5M (GSGT Version	2.0.4). This includes a hader with number of SNPs, samples.... and then the data with sample index, sample names, alleles... the first row includes the column names. This is a BPM file.
    - From this file, we can obtain the a lgen file. It is just taking the first few columns of the FinalReport. Plink has an option to read the .lgen format and convert it to PED file (see below; [link](https://www.biostars.org/p/13302/))

- A SNP map file with the physical positions of the snps.

- A sample map file with info for each individual, like the sex, ID..

It is usually recommended to prepare this data to create a ped file with Plink, which is a tool to process genetic data ([link](https://www.cog-genomics.org/plink/)), perform some filtering and QC analyses and then export as VCF ([link](https://www.biostars.org/p/210516/), [link](https://www.biostars.org/p/135156/), [link](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3066182/)) and use packages like Hail in python.

There is an interesting alternative, gtc2vcf ([link](https://github.com/freeseek/gtc2vcf), [link](https://software.broadinstitute.org/software/gtc2vcf/)), which can directly transform Ilumina reports into VCF files from command line. We are going to use Plink, though, because it is much more widely used and there are tutorials and best practice papers using it.

In particular, we are going to use the a paper about QC by Ritchie.There is a first version 2011 ([link](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3066182/)) and a second version in 2022 ([link](https://currentprotocols.onlinelibrary.wiley.com/doi/10.1002/cpz1.603)).



## Plink Installation

I have downloaded Plink ([link](https://www.cog-genomics.org/plink/)) and copied the executable ("plink") to `bin`, so we can use Plink from any place just typing `plink`. We are using Plink version 1.9 (Dec 2022).

In [2]:
%%bash

plink --version

PLINK v1.90b6.27 64-bit (10 Dec 2022)


Note that there is Plink 1.9 ([link](https://www.cog-genomics.org/plink/1.9/)) and Plink 2.0 ([link](https://www.cog-genomics.org/plink/2.0/)), these are not connected but different programs. 

This [threat](https://www.biostars.org/p/299855/#:~:text=The%20main%20difference%20is%20that,for%20a%20while%20to%20come.) of biostars explains the differences:

"The main difference is that plink 1.9 is essentially finished, while plink 2.0 is an alpha-stage program which will have significant unfinished components for a while to come. As a consequence, current development priorities for plink 2.0 are centered around things which are impossible to do with plink 1.9, such as handling multiallelic/phased variants and dosage data and reliably tracking REF/ALT alleles; while things that plink 1.9 already handles perfectly well, such as working with .ped/.map file pairs, have been deliberately deprioritized for now.

So, **you should stick to 1.9 as long as it's good enough for the jobs you need to perform. But once you need to do something outside 1.9's scope, you're likely to find that 2.0 already has the additional feature you need** (or it'll be added quickly after you ask for it)"

## Plink dummy example

Follow the general usage page ([link](https://www.cog-genomics.org/plink/1.9/general_usage)) of plink and run the toy example found in the downloaded folder.

In [3]:
%%bash

cd /media/dftortosa/Windows/Users/dftor/Documents/diego_docs/science/software/plink/plink_linux_x86_64_20221210

plink --file toy --freq --out toy_analysis

PLINK v1.90b6.27 64-bit (10 Dec 2022)          www.cog-genomics.org/plink/1.9/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to toy_analysis.log.
Options in effect:
  --file toy
  --freq
  --out toy_analysis

64162 MB RAM detected; reserving 32081 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (2 variants, 2 people).
--file: toy_analysis-temporary.bed + toy_analysis-temporary.bim +243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394%
toy_analysis-temporary.fam written.
2 variants loaded from .bim file.
2 people (2 males, 0 females) loaded from .fam.
2 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758

Explanation of the arguments

- "--file toy" tells PLINK to use the genomic data in the text files toy.ped and toy.map. You'll see several other ways to specify input data on the next page.
    - toy.map has two rows, one for each SNP ([link](https://www.cog-genomics.org/plink/1.9/formats#map)).
        - Chromosome code. PLINK 1.9 also permits contig names here, but most older programs do not.
        - Variant identifier
        - Position in morgans or centimorgans (optional; also safe to use dummy value of '0')
        - Base-pair coordinate 
        - All lines must have the same number of columns (so either no lines contain the morgans/centimorgans column, or all of them do).
    - toy.ped. Contains no header line, and one line per sample with 2V+6 fields where V is the number of variants. 
        - The first six fields are the same as those in a .fam file:
            - Family ID ('FID')
            - Within-family ID ('IID'; cannot be '0')
            - Within-family ID of father ('0' if father isn't in dataset)
            - Within-family ID of mother ('0' if mother isn't in dataset)
            - Sex code ('1' = male, '2' = female, '0' = unknown)
            - Phenotype value ('1' = control, '2' = case, '-9'/'0'/non-numeric = missing data if case/control)
        - The seventh and eighth fields are allele calls for the first variant in the .map file ('0' = no call); the 9th and 10th are allele calls for the second variant; and so on.
        - If all alleles are single-character, PLINK 1.9 will correctly parse the more compact "compound genotype" variant of this format, where each genotype call is represented as a single two-character string. This does not require the use of an additional loading flag. You can produce such a file with "--recode compound-genotypes". 
        - It is also possible to load .ped files missing some initial fields.
- --freq tells PLINK to generate an allele frequency report. The full range of supported calculations is summarized under "Main functions" in the sidebar, and the formats of all reports are described in the file formats appendix.
- --out is for the output file prefix.

So this particular combination makes PLINK calculate allele frequencies in toy.ped + toy.map, and write a report to toy_analysis.frq.

Here ([link](https://www.cog-genomics.org/plink/1.9/general_usage)) you can find information about the flag usage in plink. As an example, we are going to work with --out flag.

By default, the output files generated by PLINK all have names of the form 'plink.<one of these extensions>'. This is fine for a single run, but as soon as you make more use of PLINK, you'll start causing results from previous runs to be overwritten.

Therefore, you usually want to choose a different output file prefix for each run. --out causes 'plink' to be replaced with the prefix you provide. E.g. in the example above, "--out toy_analysis" caused PLINK to create a file named toy_analysis.frq instead of plink.frq.

Since the prefix is a required parameter, invoking --out without it will cause PLINK to quit during command line parsing:

In [4]:
%%bash

plink --file toy --freq --out

Error: Missing --out parameter.
For more information, try "plink --help <flag name>" or "plink --help | more".


PLINK v1.90b6.27 64-bit (10 Dec 2022)          www.cog-genomics.org/plink/1.9/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3


CalledProcessError: Command 'b'\nplink --file toy --freq --out\n'' returned non-zero exit status 5.

##  Exploration of a final report

Explore the final report file of one sample.

We are going to avoid the header, so we skip the first 10 rows leaving a total of 40 rows, i.e., 39 SNPs plus the columns names.

In [2]:
import pandas as pd


final_report1 = pd.read_csv("data/example_data/ILGSA24-17303_FinalReport1.txt",
                           skiprows=10, #skip rows with the header
                           delimiter="\t",
                           low_memory=False) 
    #low_memory: Internally process the file in chunks, 
    #resulting in lower memory use while parsing, 
    #but possibly mixed type inference. To ensure no mixed 
    #types either set False, or specify the type with the dtype parameter. 
final_report1

,Sample Index,Sample ID,Sample Name,SNP Index,SNP Name,Chr,Position,GT Score,GC Score,Allele1 - AB,...,Y Raw,X,Y,B Allele Freq,Log R Ratio,SNP Aux,SNP,ILMN Strand,Top Genomic Sequence,Customer Strand
0,1,7684BSAO,NaN,1,1:103380393,1,102914837,0.7987,0.8136,A,...,958,0.324,0.360,0.4849,0.3097,0,[T/C],BOT,NaN,TOP
1,1,7684BSAO,NaN,2,1:109439680,1,108897058,0.8792,0.4803,A,...,131,0.779,0.019,0.0000,0.2660,0,[A/G],TOP,NaN,TOP
2,1,7684BSAO,NaN,3,1:110198788,1,109656166,0.8780,0.9154,A,...,198,1.170,0.028,0.0000,0.0312,0,[T/C],BOT,NaN,BOT
3,1,7684BSAO,NaN,4,1:110201112,1,109658490,0.8584,0.4545,A,...,728,1.154,0.072,0.0188,-0.0746,0,[G/C],BOT,NaN,TOP
4,1,7684BSAO,NaN,5,1:110201667,1,109659045,0.7815,0.7185,B,...,3269,0.120,1.291,0.9797,-0.0749,0,[T/C],BOT,NaN,BOT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654022,1,7684BSAO,NaN,654023,Y:8804321-A-C,Y,8936280,0.7229,0.4350,A,...,157,0.403,0.039,0.0000,-1.2565,0,[T/G],BOT,NaN,TOP
654023,1,7684BSAO,NaN,654024,Y:8852748-A-C,Y,8984707,0.8575,0.4533,A,...,147,0.730,0.028,0.0000,-0.3919,0,[T/G],BOT,NaN,TOP
654024,1,7684BSAO,NaN,654025,Y:8894070-A-T,Y,9026029,0.8175,0.4061,B,...,3099,0.017,1.436,1.0000,0.0729,0,[A/T],TOP,NaN,BOT
654025,1,7684BSAO,NaN,654026,Y:9145267-T-C,Y,9307658,0.7893,0.2966,A,...,201,0.334,0.059,0.0178,-1.3112,0,[T/C],BOT,NaN,BOT


See columns names (see this [tutorial](https://www.youtube.com/watch?v=_-Gu9hO8aFM&ab_channel=GenomicsBootCamp)):

In [18]:
final_report1.columns

Index(['Sample Index', 'Sample ID', 'Sample Name', 'SNP Index', 'SNP Name',
       'Chr', 'Position', 'GT Score', 'GC Score', 'Allele1 - AB',
       'Allele2 - AB', 'Allele1 - Top', 'Allele2 - Top', 'Allele1 - Forward',
       'Allele2 - Forward', 'Allele1 - Design', 'Allele2 - Design', 'Theta',
       'R', 'X Raw', 'Y Raw', 'X', 'Y', 'B Allele Freq', 'Log R Ratio',
       'SNP Aux', 'SNP', 'ILMN Strand', 'Top Genomic Sequence',
       'Customer Strand'],
      dtype='object')

**Sample identification**

The first three columns are about the sample. At least in our case, we have 1 final report for each sample, i.e., each individual (see below), so these columns should have only one value in the entire file. The same sample index and ID. We do not have sample name.

In [3]:
import numpy as np

print(f'Unique cases are 1? {len(np.unique(final_report1["Sample Index"]))==1}')
print(f'Unique cases are 1? {len(np.unique(final_report1["Sample ID"]))==1}')
print(f'Unique cases are 1? {len(np.unique(final_report1["Sample Name"]))==1}')

Unique cases are 1? True
Unique cases are 1? True
Unique cases are 1? True


**SNPs identification**

Then, we have the SNP indexes and names. We should have a different value for each row, i.e., the number of unique cases should be the same than the number of rows, i.e., no duplicates. Therefore, **we have as many rows as SNPs**.

In [36]:
print(f'Unique cases are the number of rows? {len(np.unique(final_report1["SNP Index"]))==final_report1.shape[0]}')
print(f'Unique cases are the number of rows? {len(np.unique(final_report1["SNP Name"]))==final_report1.shape[0]}')

Unique cases are the number of rows? True
Unique cases are the number of rows? True


**Chromosome data**

Then, we have the chromosome. We have some extrange cases like chromosome 0, XY and MT

In [28]:
np.unique(final_report1["Chr"])

array(['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '2', '20', '21', '22', '3', '4', '5', '6', '7', '8', '9',
       'MT', 'X', 'XY', 'Y'], dtype=object)

We have 210 SNPs out 600K that have chromosome zero, and they also have position zero. These seem to be outdated SNPs as they do not have any physical position.

In [49]:
f'SNPs with chromosome zero are less than 1000? {final_report1.loc[final_report1["Chr"] == "0"].shape[0] < 1000}'

'SNPs with chromosome zero are less than 1000? True'

In [50]:
f'Chromosome zero SNPs have also position zero? {np.unique(final_report1.loc[final_report1["Chr"] == "0", "Position"]) == 0}'

'Chromosome zero SNPs have also position zero? [ True]'

We have also SNPs in chromosome XY.

In [48]:
final_report1.loc[final_report1["Chr"] == "XY"]

,Sample Index,Sample ID,Sample Name,SNP Index,SNP Name,Chr,Position,GT Score,GC Score,Allele1 - AB,...,Y Raw,X,Y,B Allele Freq,Log R Ratio,SNP Aux,SNP,ILMN Strand,Top Genomic Sequence,Customer Strand
14065,1,7684BSAO,NaN,14066,exm2262791,XY,2675970,0.8615,0.8977,A,...,192,1.410,0.028,0.0000,0.1230,0,[T/G],BOT,NaN,TOP
16165,1,7684BSAO,NaN,16166,exm2273223,XY,265112,0.7593,0.7487,A,...,279,1.814,0.055,0.0000,0.2450,0,[A/G],TOP,NaN,BOT
16166,1,7684BSAO,NaN,16167,exm2273224,XY,1310262,0.8552,0.8905,A,...,204,1.235,0.040,0.0000,0.1650,0,[T/C],BOT,NaN,BOT
312826,1,7684BSAO,NaN,312827,JHU_X.1211573,XY,1111421,0.7731,0.7040,B,...,4575,0.083,1.829,0.9857,-0.1434,0,[A/G],TOP,NaN,TOP
312832,1,7684BSAO,NaN,312833,JHU_X.1215195,XY,1115043,0.7880,0.7967,A,...,137,0.608,0.027,0.0000,-0.3454,0,[T/C],BOT,NaN,TOP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647731,1,7684BSAO,NaN,647732,rs9786294,XY,92802202,0.7648,0.6893,B,...,3295,0.114,1.325,0.9915,0.0394,0,[T/G],BOT,NaN,BOT
647740,1,7684BSAO,NaN,647741,rs9786334,XY,90826461,0.8043,0.7570,A,...,221,1.006,0.051,0.0172,-0.3718,0,[A/G],TOP,NaN,BOT
647776,1,7684BSAO,NaN,647777,rs9786468,XY,91577410,0.8548,0.8337,B,...,1530,0.083,0.606,0.9670,-0.0696,0,[A/G],TOP,NaN,TOP
648034,1,7684BSAO,NaN,648035,rs9792836,XY,92590088,0.7972,0.3833,B,...,3397,0.083,1.367,0.9906,-0.2638,0,[A/G],TOP,NaN,BOT


In [54]:
f'The number of XY SNPs is less than 1000: {final_report1.loc[final_report1["Chr"] == "XY"].shape[0]<1000}'

'The number of XY SNPs is less than 1000: True'

These seems to be pseudoautosomal regions (PAR), i.e., homologous sequences of nucleotides on the X and Y chromosomes. Therefore, this should be ok, the error is to put them in X or XY while the convention is to set them as XY ([link; section Sex-chromosome marker annotation](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5100670/)).

We have also mitochondrial SNPs.

In [51]:
final_report1.loc[final_report1["Chr"] == "MT"]

,Sample Index,Sample ID,Sample Name,SNP Index,SNP Name,Chr,Position,GT Score,GC Score,Allele1 - AB,...,Y Raw,X,Y,B Allele Freq,Log R Ratio,SNP Aux,SNP,ILMN Strand,Top Genomic Sequence,Customer Strand
8158,1,7684BSAO,NaN,8159,200610-1,MT,2755,0.6796,0.5358,A,...,775,3.385,0.219,0.0000,0.0727,0,[T/C],BOT,NaN,TOP
8159,1,7684BSAO,NaN,8160,200610-100,MT,15172,0.6310,0.2317,B,...,4100,0.432,1.586,1.0000,-0.2949,0,[T/C],BOT,NaN,TOP
8160,1,7684BSAO,NaN,8161,200610-102,MT,125,0.7739,0.3583,A,...,299,1.618,0.071,0.0000,-0.0262,0,[A/G],TOP,NaN,BOT
8161,1,7684BSAO,NaN,8162,200610-105,MT,236,0.6705,0.5197,A,...,371,3.063,0.067,0.0000,0.1016,0,[A/G],TOP,NaN,BOT
8162,1,7684BSAO,NaN,8163,200610-110,MT,3027,0.7420,0.6486,A,...,368,2.516,0.081,0.0000,0.1526,0,[A/G],TOP,NaN,BOT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545420,1,7684BSAO,NaN,545421,rs62581340,MT,16356,0.6619,0.5046,B,...,1913,0.178,0.760,1.0000,0.0146,0,[T/C],BOT,NaN,BOT
545421,1,7684BSAO,NaN,545422,rs62581341,MT,16362,0.6257,0.4430,A,...,407,0.464,0.140,0.0804,-1.7221,0,[T/C],BOT,NaN,BOT
591382,1,7684BSAO,NaN,591383,rs75214962,MT,11197,0.7111,0.2970,B,...,2214,0.179,0.883,0.9925,-0.3114,0,[T/C],BOT,NaN,TOP
593585,1,7684BSAO,NaN,593586,rs75577869,MT,13020,0.6882,0.5512,A,...,400,1.397,0.115,0.0000,-0.1041,0,[T/C],BOT,NaN,TOP


In [55]:
f'The number of mito SNPs is less than 1500: {final_report1.loc[final_report1["Chr"] == "MT"].shape[0]<1500}'

'The number of mito SNPs is less than 1500: True'

**Position**

Then, we have the position, which is an integer.

In [18]:
print(f'The type of Position is integer? {final_report1["Position"].dtype == "int64"}')
print(f'There are no NAs? {final_report1.loc[final_report1["Position"].isna()].shape[0] == 0}')

The type of Position is integer? True
There are no NAs? True


**Quality measures**

Then, we have the GT and GC scores, which are quality measures.

The most important QC parameter is the GenTrain score. The GenTrain score is computed from the GenTrain 2.0 clustering algorithm. It is a measurement of SNP calling quality, ranging from 0 to 1, with higher value meaning better quality ([link](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6171493/)).

In [25]:
print(f'The type of GT score is integer? {final_report1["GT Score"].dtype == "float64"}')
print(f'There are no NAs? {final_report1.loc[final_report1["GT Score"].isna()].shape[0] == 0}')
print(f'Min is equal of higher than 0? {np.min(final_report1["GT Score"]) >= 0}')
print(f'Max is equal or lower than 1? {np.max(final_report1["GT Score"]) <= 1}')

The type of GT score is integer? True
There are no NAs? True
Min is equal of higher than 0? True
Max is equal or lower than 1? True


The second most important QC parameter is the cluster separation score, which measures how well the AA, AB and BB clusters are separated. The cluster separation score also ranges from 0 to 1, with higher meaning better (more separation; [link](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6171493/)). 

In the link cited, in figure 4, you can see several cases for specific snps where the separation between clusters (AA, AB and BB) is not clear (left panels). Meaning that there individuals are not clearly differentiated for this SNP. With realignement they get a better separation and a better score.

NOT PRESENT IN MY DATASET

The third most important QC parameter is call frequency, which measures the percentage of samples with successful calls for that SNP. The call frequency also ranges from 0 to 1, with higher meaning more samples have successful calls for this SNP ([link](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6171493/)).

In [27]:
print(f'The type of GT score is integer? {final_report1["GC Score"].dtype == "float64"}')
print(f'There are no NAs? {final_report1.loc[final_report1["GC Score"].isna()].shape[0] == 0}')
print(f'Min is equal of higher than 0? {np.min(final_report1["GC Score"]) >= 0}')
print(f'Max is equal or lower than 1? {np.max(final_report1["GC Score"]) <= 1}')

The type of GT score is integer? True
There are no NAs? True
Min is equal of higher than 0? True
Max is equal or lower than 1? True


**Log R ratio and BAF**

These metrics could be used to detect copy number variation (e.g., deletions) without having whole genome sequencing ([Nandolo et al 2018](https://www.youtube.com/watch?v=_-Gu9hO8aFM&ab_channel=GenomicsBootCamp)).

**Alleles**

One of the biggest weakness of the Illumina genotyping array design is Illumina’s definition of strand. As DNA is double-stranded, significant SNPs from GWAS need to be presented with their strand information to properly report the risk alleles. This unfortunately has not been a standard practice. The most intuitive definition of strand is to use the human genome reference as the forward strand. Defying logic, Illumina introduced a more convoluted definition of strand: top and bottom [25], which has caused great confusion with reference to the forward and reverse strand [26, 27] ([link](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6171493/)).

DNA strand designations ([link](https://www.biostars.org/p/4885/)):
- Customer Strand:Same as the Source strand. For custom content, it is the strand submitted by the customer for probe design.
- ILMN Strand, a.k.a. Design Strand: The strand used by Illumina to design probes based on thermodynamic stability and locus specificity according to NCBI BLAST. For this reason, it can differ from the Customer/ Source strand.
- Forward/Reverse (Fwd/Rev) Strand: Used by dbSNP, **Fwd/Rev designations can change with NCBI Genome Build updates, so Genome Build must be specified when reporting Fwd/Rev strands**. 1. For SNPs in standard array products, Fwd strand = Source strand, and originates from dbSNP. 2. For custom array product SNPs without rsid’s, the customer can identify the Source strand as Fwd or Rev, based on their own criteria. Illumina custom product files use the customer’s Fwd/Rev designations. Note: The Fwd strand, as identified in Illumina standard product files, should not be confused with Plus (+) strand, which HapMap interchangeably calls the “forward strand.”
- Plus/Minus (+/-) Strand: The standard designation for all eukaryotic organisms used by HapMapand 1000 Genomes Project. The 5′ end of the (+) strand is at the tip of the short arm (p arm) of the chromosome and the 5′ end of the (-) strand is at the tip of the long arm (q arm). (+/-) designations can change with NCBI Genome Build updates, so Genome Build must be specified when reporting (+/-) strands.
- Source Strand: Same as the Customer strand. The strand submitted to the Illumina designer for probe design. 1. For standard SNPs, it is the Fwd strand as reported in the source database (i.e., dbSNP). 2. Custom content can be reported as rsid’s or as the DNA sequences or chromosomal regions, depending on the format submitted by the customer.
- Top/Bottom (Top/Bot) Strand:Top/Bot nomenclature was developed by Illumina using sequence-based context to assign strand designations that does not change regardless of database or genome assembly used. (e.g., depending on the NCBI Genome Build referenced, strand and allele designations can change). Top/Bot is not directly related to Fwd/Rev or (+/-).Top/Bot strand is determined by examining the SNP and the surrounding DNA sequence and it only applies to SNPs with two possible alleles. See the Top/Bot A/B Allele bulletin for more details ([link](https://www.illumina.com/documents/products/technotes/technote_topbot.pdf), [link](https://support.illumina.com/bulletins/2017/06/how-to-interpret-dna-strand-and-allele-information-for-infinium-.html)).

See the different allele names for some SNPs with rs number:

In [20]:
final_report1.loc[final_report1["SNP Name"].apply(lambda x: "rs" in x), ['SNP Name', 
                                                                         'Allele1 - AB', 
                                                                         'Allele2 - AB', 
                                                                         'Allele1 - Top', 
                                                                         'Allele2 - Top', 
                                                                         'Allele1 - Forward',
                                                                         'Allele2 - Forward', 
                                                                         'Allele1 - Design', 
                                                                         'Allele2 - Design',
                                                                         'ILMN Strand', 
                                                                         'SNP']]

,SNP Name,Allele1 - AB,Allele2 - AB,Allele1 - Top,Allele2 - Top,Allele1 - Forward,Allele2 - Forward,Allele1 - Design,Allele2 - Design,ILMN Strand,SNP
12303,chr10-96521657:rs12248560-TC,B,B,G,G,C,C,C,C,BOT,[T/C]
12304,chr10-96535210:rs72552267,B,B,G,G,G,G,G,G,TOP,[A/G]
12305,chr10-96541616:rs4244285-AG,B,B,G,G,G,G,C,C,BOT,[T/C]
12335,chr16-31107689:rs9923231-CT,A,B,A,G,T,C,A,G,TOP,[A/G]
12369,chr22-42525044:rs1135824,A,A,A,A,T,T,T,T,BOT,[T/C]
...,...,...,...,...,...,...,...,...,...,...,...
652993,rs9999844,B,B,G,G,G,G,C,C,BOT,[T/C]
652994,rs9999847,B,B,G,G,C,C,G,G,TOP,[A/G]
652995,rs999986,A,B,A,C,T,G,T,G,BOT,[T/G]
652996,rs999989,A,A,A,A,T,T,T,T,BOT,[T/G]


For example, rs9999844 has for foward, the aleles indicated in dbSNP. But this does not work always. For example, according to dbSNP, rs999994 is C/T, but we have G.

We will use Allele-Foward in general, but we will have to check the alleles at some point, maybe after selecting SNPs for the polygenic score?

**Other columns**

There still other columns, probably related to the plots in the report of illumina, but we are not going to explore them for now unless the QC tutorials make use of them.

In [22]:
final_report1.columns

Index(['Sample Index', 'Sample ID', 'Sample Name', 'SNP Index', 'SNP Name',
       'Chr', 'Position', 'GT Score', 'GC Score', 'Allele1 - AB',
       'Allele2 - AB', 'Allele1 - Top', 'Allele2 - Top', 'Allele1 - Forward',
       'Allele2 - Forward', 'Allele1 - Design', 'Allele2 - Design', 'Theta',
       'R', 'X Raw', 'Y Raw', 'X', 'Y', 'B Allele Freq', 'Log R Ratio',
       'SNP Aux', 'SNP', 'ILMN Strand', 'Top Genomic Sequence',
       'Customer Strand'],
      dtype='object')

## Load and explore the sample and snp maps

We need these maps in order to generate the inputs for Plink (see below).

**SNP map**

In [4]:
snp_map = pd.read_csv("data/example_data/SNP_Map.txt",
                      delimiter="\t", 
                      header=0,
                      low_memory=False) 
    #low_memory: Internally process the file in chunks, 
    #resulting in lower memory use while parsing, 
    #but possibly mixed type inference. To ensure no mixed 
    #types either set False, or specify the type with the dtype parameter. 
snp_map

,Index,Name,Chromosome,Position,GenTrain Score,SNP,ILMN Strand,Customer Strand,NormID
0,1,1:103380393,1,102914837,0.7987,[T/C],BOT,TOP,28
1,2,1:109439680,1,108897058,0.8792,[A/G],TOP,TOP,1
2,3,1:110198788,1,109656166,0.8780,[T/C],BOT,BOT,4
3,4,1:110201112,1,109658490,0.8584,[G/C],BOT,TOP,101
4,5,1:110201667,1,109659045,0.7815,[T/C],BOT,BOT,1
...,...,...,...,...,...,...,...,...,...
654022,654023,Y:8804321-A-C,Y,8936280,0.7229,[T/G],BOT,TOP,3
654023,654024,Y:8852748-A-C,Y,8984707,0.8575,[T/G],BOT,TOP,3
654024,654025,Y:8894070-A-T,Y,9026029,0.8175,[A/T],TOP,BOT,203
654025,654026,Y:9145267-T-C,Y,9307658,0.7893,[T/C],BOT,BOT,3


Check we have the same number of SNPs in the final report and the SNP map

In [26]:
f'Number of SNP matches between SNP map and final report? {snp_map.shape[0] == final_report1.shape[0]}'

'Number of SNP matches between SNP map and final report? True'

Therefore, we have a row per SNP, having each SNP an Index and a Name

In [35]:
print(f'Unique cases are the number of rows? {len(np.unique(snp_map["Index"]))==snp_map.shape[0]}')
print(f'Unique cases are the number of rows? {len(np.unique(snp_map["Name"]))==snp_map.shape[0]}')

Unique cases are 1? True
Unique cases are 1? True


Each SNP has its chromose, having autosomals but also MT, zero cases and sex chromosomes..

In [38]:
np.unique(snp_map["Chromosome"])

array(['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '2', '20', '21', '22', '3', '4', '5', '6', '7', '8', '9',
       'MT', 'X', 'XY', 'Y'], dtype=object)

We have SNPS that have chromosome zero, and they also have position zero. They seem to be outdated SNPs.

In [39]:
f'SNPs with chromosome zero are less than 1000? {snp_map.loc[snp_map["Chromosome"] == "0"].shape[0] < 1000}'

'SNPs with chromosome zero are less than 1000? True'

In [50]:
f'Chromosome zero SNPs have also position zero? {np.unique(snp_map.loc[snp_map["Chromosome"] == "0", "Position"]) == 0}'

'Chromosome zero SNPs have also position zero? [ True]'

We have also SNPs in chromosome XY.

In [40]:
snp_map.loc[snp_map["Chromosome"] == "XY"]

,Index,Name,Chromosome,Position,GenTrain Score,SNP,ILMN Strand,Customer Strand,NormID
14065,14066,exm2262791,XY,2675970,0.8615,[T/G],BOT,TOP,6
16165,16166,exm2273223,XY,265112,0.7593,[A/G],TOP,BOT,6
16166,16167,exm2273224,XY,1310262,0.8552,[T/C],BOT,BOT,1
312826,312827,JHU_X.1211573,XY,1111421,0.7731,[A/G],TOP,TOP,34
312832,312833,JHU_X.1215195,XY,1115043,0.7880,[T/C],BOT,TOP,34
...,...,...,...,...,...,...,...,...,...
647731,647732,rs9786294,XY,92802202,0.7648,[T/G],BOT,BOT,3
647740,647741,rs9786334,XY,90826461,0.8043,[A/G],TOP,BOT,3
647776,647777,rs9786468,XY,91577410,0.8548,[A/G],TOP,TOP,3
648034,648035,rs9792836,XY,92590088,0.7972,[A/G],TOP,BOT,3


In [41]:
f'The number of XY SNPs is less than 1000: {snp_map.loc[snp_map["Chromosome"] == "XY"].shape[0]<1000}'

'The number of XY SNPs is less than 1000: True'

These seems to be pseudoautosomal regions (PAR), i.e., homologous sequences of nucleotides on the X and Y chromosomes. Therefore, this should be ok, the error is to put them in X or XY while the convention is to set them as XY ([link; section Sex-chromosome marker annotation](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5100670/)).

We have also mitochondrial SNPs.

In [43]:
snp_map.loc[snp_map["Chromosome"] == "MT"]

,Index,Name,Chromosome,Position,GenTrain Score,SNP,ILMN Strand,Customer Strand,NormID
8158,8159,200610-1,MT,2755,0.6796,[T/C],BOT,TOP,6
8159,8160,200610-100,MT,15172,0.6310,[T/C],BOT,TOP,6
8160,8161,200610-102,MT,125,0.7739,[A/G],TOP,BOT,1
8161,8162,200610-105,MT,236,0.6705,[A/G],TOP,BOT,6
8162,8163,200610-110,MT,3027,0.7420,[A/G],TOP,BOT,1
...,...,...,...,...,...,...,...,...,...
545420,545421,rs62581340,MT,16356,0.6619,[T/C],BOT,BOT,3
545421,545422,rs62581341,MT,16362,0.6257,[T/C],BOT,BOT,203
591382,591383,rs75214962,MT,11197,0.7111,[T/C],BOT,TOP,3
593585,593586,rs75577869,MT,13020,0.6882,[T/C],BOT,TOP,3


In [44]:
f'The number of mito SNPs is less than 1500: {snp_map.loc[snp_map["Chromosome"] == "MT"].shape[0]<1500}'

'The number of mito SNPs is less than 1500: True'

Then, we have GenTrain score for each SNP.

In [53]:
f'There is no NA? {all(snp_map["GenTrain Score"].notna())}'

'There is no NA? True'

In [56]:
f'Float? {snp_map["GenTrain Score"].dtype == "float64"}'

'Float? True'

In [57]:
print(snp_map["GenTrain Score"].describe())

count    654027.000000
mean          0.807331
std           0.087710
min           0.000000
25%           0.782200
50%           0.817900
75%           0.856800
max           0.965100
Name: GenTrain Score, dtype: float64


There is also allele and strand information. Not sure what strand is shown, but we are going to use foward in final report anyways.

The SNP I checked with mismatch between dbSNP and my database has also mismatch here.

In [60]:
snp_map.loc[snp_map["Name"] == "rs999994"]

,Index,Name,Chromosome,Position,GenTrain Score,SNP,ILMN Strand,Customer Strand,NormID
652997,652998,rs999994,10,76975903,0.8048,[A/G],TOP,TOP,4


**Sample map**

In [5]:
sample_map = pd.read_csv("data/example_data/Sample_Map.txt",
                      delimiter="\t", 
                      header=0,
                      low_memory=False) 
    #low_memory: Internally process the file in chunks, 
    #resulting in lower memory use while parsing, 
    #but possibly mixed type inference. To ensure no mixed 
    #types either set False, or specify the type with the dtype parameter. 
sample_map

,Index,Name,ID,Gender,Plate,Well,Group,Parent1,Parent2,Replicate,SentrixPosition
0,1,NaN,7684BSAO,Male,NaN,NaN,NaN,NaN,NaN,NaN,205422690011_R01C01
1,2,NaN,7691CPSO,Male,NaN,NaN,NaN,NaN,NaN,NaN,205422690011_R03C01
2,3,NaN,7688MRFO,Male,NaN,NaN,NaN,NaN,NaN,NaN,205422690011_R05C01
3,4,NaN,7698ATMO,Female,NaN,NaN,NaN,NaN,NaN,NaN,205422690011_R07C01
4,5,NaN,7695NKSO,Male,NaN,NaN,NaN,NaN,NaN,NaN,205422690011_R09C01
...,...,...,...,...,...,...,...,...,...,...,...
211,212,NaN,6897TSSS,Male,NaN,NaN,NaN,NaN,NaN,NaN,205422690123_R04C02
212,213,NaN,6997EDFS,Female,NaN,NaN,NaN,NaN,NaN,NaN,205422690123_R06C02
213,214,NaN,6900CBNS,Male,NaN,NaN,NaN,NaN,NaN,NaN,205422690123_R08C02
214,215,NaN,6991HBJS,Male,NaN,NaN,NaN,NaN,NaN,NaN,205422690123_R10C02


The number of samples should be 216 because this is the number of final reports present in the first batch

In [62]:
sample_map.shape[0] == 216

True

We have index and ID for each sample

In [63]:
print(f'Unique cases are the number of rows? {len(np.unique(sample_map["Index"]))==sample_map.shape[0]}')
print(f'Unique cases are the number of rows? {len(np.unique(sample_map["ID"]))==sample_map.shape[0]}')

Unique cases are the number of rows? True
Unique cases are the number of rows? True


We have gender information, whith male, female and Unknown. This is estimated from illumina, not from the study!!

In [65]:
np.unique(sample_map["Gender"])

array(['Female', 'Male', 'Unknown'], dtype=object)

There are three individuals with unknown sex according to Illumina data. These are indeed the three problematic individuals according to the logR value, which is above the Illumina expecation of 0.3.

In [66]:
sample_map.loc[sample_map["Gender"] == "Unknown"]

,Index,Name,ID,Gender,Plate,Well,Group,Parent1,Parent2,Replicate,SentrixPosition
37,38,NaN,7699ZSSO,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,205422690078_R03C02
70,71,NaN,3400SMJJ,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,205422690083_R10C02
149,150,NaN,2498STAA,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,205422690121_R11C01


In [71]:
f'Unknown sex is below 10? {sample_map.loc[(sample_map["Gender"] == "Unknown")].shape[0] < 10}'

'Unknown sex is below 10? True'

Then, we have columns for ID of the parents, but this is not the case in our study.

## Load phenotype data

This include reported sex and VO2 max data.

In [5]:
pheno_data = pd.read_csv("data/pheno_data/combact gene DNA GWAS 23062022_all_dna_samples.csv",
                      delimiter=",", 
                      header=0,
                      low_memory=False) 
    #low_memory: Internally process the file in chunks, 
    #resulting in lower memory use while parsing, 
    #but possibly mixed type inference. To ensure no mixed 
    #types either set False, or specify the type with the dtype parameter. 
pheno_data

,Gender,Age,Week 1 Body Mass,Week 8 Body Mass,Week 1 Beep test,Week 8 beep test,Week 1 Pred VO2max,Week 8 Pred VO2max,AGRF code
0,M,25.000000,NaN,75.0,9.6,10.3,45.2,47.7,8244TCOJ
1,M,43.720548,79.7,80.8,8.6,9.1,41.8,43.6,6976MCJS
2,M,35.049315,86.2,86.2,10.2,10.7,47.4,49.0,3484JSJJ
3,F,23.561644,70.0,70.0,8.1,9.1,40.2,43.6,1389HSNM
4,M,26.164384,96.2,93.3,10.2,12.1,47.4,54.0,7593DBAO
...,...,...,...,...,...,...,...,...,...
1459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2471AWMA
1460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6897TSSS
1461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6899DWOS
1462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6296TONJ


We have gender (F/M) with some NAs

In [12]:
pheno_data["Gender"].describe()

count     1422
unique       2
top          M
freq      1228
Name: Gender, dtype: object

In [20]:
f'NAs for sex: {pheno_data[pheno_data["Gender"].isna()].shape[0]}'

'NAs for sex: 42'

Also age, which is a float, as shown in the paper

In [21]:
pheno_data["Age"].describe()

count    1422.000000
mean       21.969568
std         5.065370
min        17.000000
25%        18.767808
50%        20.163014
75%        23.166438
max        56.254237
Name: Age, dtype: float64

Then, different body mass and cardiorespiratory fitness variables.

NOT CHECKED FOR NOW.

We also have the ID of the sample

In [36]:
pheno_data["AGRF code"]

0       8244TCOJ
1       6976MCJS
2       3484JSJJ
3       1389HSNM
4       7593DBAO
          ...   
1459    2471AWMA
1460    6897TSSS
1461    6899DWOS
1462    6296TONJ
1463    7799BTDO
Name: AGRF code, Length: 1464, dtype: object

This code is the ID of the samples in the sample map of illumina.

We have almost all samples of the first batch included in the pheno data, we only lack one individual.

In [39]:
pheno_data[pheno_data["AGRF code"].isin(sample_map["ID"])].shape[0]

215

## Conversion of final report to Plink inputs

We are going to convert the illumina report (BPM file) to lgen file that can be used as input in Plink. This seems to be trivial ([link](https://www.biostars.org/p/51928/)), but we are going to use a tutorial just in case ([link](https://www.youtube.com/watch?v=5_pNby7l2dE&t=1s&ab_channel=GenomicsBootCamp), [link](https://pastebin.com/pzgw7JVp)).

The final report has 1 line for the SNP of one individual. We have separated final reports for each individual. This is close to the lgen format of plink.

Therefore, our goal is to create lgen, also the fam and map files required to load the lgen in Plink. Some information might be missing in the final report, so you need to replace them.

**lgen file [plink info](https://www.cog-genomics.org/plink/1.9/formats#lgen)**

A text file with no header line, and one line per genotype call (or just not-homozygous-major calls if 'lgen-ref' was invoked) usually with the following five fields:

- Family ID
- Within-family ID
- Variant identifier
- Allele call 1 ('0' for missing)
- Allele call 2

As we have each sample in a separated final report, we need to bind them in order to get all the genotype calls

Get first the paths for each final report

In [60]:
import glob
list_reports_files_full_path = glob.glob("data/example_data/ILGSA24-17303_FinalReport*") 
    #I prefer using the glob module, as it does pattern matching and expansion.
        #https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
list_reports_files_full_path

['data/example_data/ILGSA24-17303_FinalReport1.txt',
 'data/example_data/ILGSA24-17303_FinalReport2.txt']

Load the data.frame selecting only the colums needed for the lgen file

In [73]:
list_df_reports = [pd.read_csv(path,
                               skiprows=10, #skip first 10 rows to get only the data
                               usecols=["Sample ID", #load only a subset of the columns
                                        "SNP Name", 
                                        "Allele1 - Forward", 
                                        "Allele2 - Forward"],                               
                               header=0,
                               sep='\t',
                               low_memory=False) #to avoid problems with mixed types
                   for path in list_reports_files_full_path]
#for the 1000 files, we should parallelize

You can see how the selection of less columns decreases the size using the first final_report as example. We reduce the size almost 6 times.

In [51]:
import sys

#get the size of final_report1 as a whole and then to only the selected columns
#divide by 10^6 to get MBs
print(sys.getsizeof(final_report1)/(10**6))
print(sys.getsizeof(final_report1[["Sample ID", "SNP Name", "Allele1 - Forward", "Allele2 - Forward"]])/(10**6))

631.959649
163.38272


The rest of necessary columns like sample ID or sample SNP and position/chromsome are just repeated across final_reports but in the sample and SNP map are not duplicated, so we can just use these smaller files to get these required columns. See next sections.

We get a list with one DF per final report

In [76]:
list_df_reports[0]

,Sample ID,SNP Name,Allele1 - Forward,Allele2 - Forward
0,7684BSAO,1:103380393,A,G
1,7684BSAO,1:109439680,A,A
2,7684BSAO,1:110198788,T,T
3,7684BSAO,1:110201112,C,C
4,7684BSAO,1:110201667,C,C
...,...,...,...,...
654022,7684BSAO,Y:8804321-A-C,A,A
654023,7684BSAO,Y:8852748-A-C,A,A
654024,7684BSAO,Y:8894070-A-T,A,A
654025,7684BSAO,Y:9145267-T-C,T,T


In [29]:
f'Do we have as many DFs as paths we got? {len(list_df_reports) == len(list_reports_files_full_path)}'

'Do we have as many DFs as paths we got? True'

Concatenate, because we have one row per SNP and sample, so we can just concateneate DFs.

In [132]:
full_final_report = pd.concat(objs=list_df_reports, #list DFs
                              axis=0, #concatenate along rows
                              ignore_index=True) #clear the existing index and reset it

We should have as many rows as the total sum of rows across the list of DFs

In [39]:
f'Do we have all the genotype calls? {full_final_report.shape[0] == sum([element.shape[0] for element in list_df_reports])}'

'Do we have all the genotype calls? True'

In [133]:
full_final_report

,Sample ID,SNP Name,Allele1 - Forward,Allele2 - Forward
0,7684BSAO,1:103380393,A,G
1,7684BSAO,1:109439680,A,A
2,7684BSAO,1:110198788,T,T
3,7684BSAO,1:110201112,C,C
4,7684BSAO,1:110201667,C,C
...,...,...,...,...
1308049,7691CPSO,Y:8804321-A-C,A,A
1308050,7691CPSO,Y:8852748-A-C,A,A
1308051,7691CPSO,Y:8894070-A-T,A,A
1308052,7691CPSO,Y:9145267-T-C,T,T


We have I and D as genotype calls, we will have to check that

In [144]:
np.unique(full_final_report["Allele1 - Forward"])

array(['-', 'A', 'C', 'D', 'G', 'I', 'T'], dtype=object)

Make a copy, using copy(), so modifications to the data or indices of the copy will not be reflected in the original object

In [145]:
lgen_file_raw = full_final_report.copy()

Change to "0" those genotype calls with "--" to match the format of Plink

In [146]:
lgen_file_raw.loc[(lgen_file_raw["Allele1 - Forward"] == "-") | 
                      (lgen_file_raw["Allele1 - Forward"] == "--"), 
                      "Allele1 - Forward"] = "0"

In [147]:
lgen_file_raw.loc[(lgen_file_raw["Allele2 - Forward"] == "-") | 
                      (lgen_file_raw["Allele2 - Forward"] == "--"), 
                      "Allele2 - Forward"] = "0"

Check we do not have "--" or "-" in alleles

In [148]:
print("-" not in np.unique(lgen_raw["Allele1 - Forward"]))
print("--" not in np.unique(lgen_raw["Allele1 - Forward"]))
print("-" not in np.unique(lgen_raw["Allele2 - Forward"]))
print("--" not in np.unique(lgen_raw["Allele2 - Forward"]))

True
True
True
True


Add additional columns that are required for lgen files

In [149]:
lgen_file_raw["FID"] = "combat"

In [159]:
lgen_file_raw

,Sample ID,SNP Name,Allele1 - Forward,Allele2 - Forward,FID
0,7684BSAO,1:103380393,A,G,combat
1,7684BSAO,1:109439680,A,A,combat
2,7684BSAO,1:110198788,T,T,combat
3,7684BSAO,1:110201112,C,C,combat
4,7684BSAO,1:110201667,C,C,combat
...,...,...,...,...,...
1308049,7691CPSO,Y:8804321-A-C,A,A,combat
1308050,7691CPSO,Y:8852748-A-C,A,A,combat
1308051,7691CPSO,Y:8894070-A-T,A,A,combat
1308052,7691CPSO,Y:9145267-T-C,T,T,combat


Reorder the columns

In [160]:
lgen_file = lgen_file_raw[["FID", "Sample ID", "SNP Name", "Allele1 - Forward", "Allele2 - Forward"]]

In [161]:
lgen_file

,FID,Sample ID,SNP Name,Allele1 - Forward,Allele2 - Forward
0,combat,7684BSAO,1:103380393,A,G
1,combat,7684BSAO,1:109439680,A,A
2,combat,7684BSAO,1:110198788,T,T
3,combat,7684BSAO,1:110201112,C,C
4,combat,7684BSAO,1:110201667,C,C
...,...,...,...,...,...
1308049,combat,7691CPSO,Y:8804321-A-C,A,A
1308050,combat,7691CPSO,Y:8852748-A-C,A,A
1308051,combat,7691CPSO,Y:8894070-A-T,A,A
1308052,combat,7691CPSO,Y:9145267-T-C,T,T


Save without header:

In [166]:
lgen_file.to_csv("data/plink_inputs_example/batch1_example.lgen",
                 sep="\t",
                 header=None,
                 index=False)



**Fam file ([PLINK sample information file](https://www.cog-genomics.org/plink/1.9/formats#fam))**

A text file with no header line, and one line per sample with the following six fields:

- Family ID ('FID')
- Within-family ID ('IID'; cannot be '0')
- Within-family ID of father ('0' if father isn't in dataset)
- Within-family ID of mother ('0' if mother isn't in dataset)
- Sex code ('1' = male, '2' = female, '0' = unknown)
- Phenotype value ('1' = control, '2' = case, '-9'/'0'/non-numeric = missing data if case/control)

We can get the self-reported sex and sample ID from the pheno data.

In [167]:
fam_file_raw = pheno_data.loc[:, ["Gender", "AGRF code"]]
fam_file_raw

,Gender,AGRF code
0,M,8244TCOJ
1,M,6976MCJS
2,M,3484JSJJ
3,F,1389HSNM
4,M,7593DBAO
...,...,...
1459,NaN,2471AWMA
1460,NaN,6897TSSS
1461,NaN,6899DWOS
1462,NaN,6296TONJ


Codify the sex variable following plink notation

In [168]:
fam_file_raw.loc[fam_file_raw["Gender"].isna(), "Gender"] = "0"
fam_file_raw.loc[fam_file_raw["Gender"] == "M", "Gender"] = "1"
fam_file_raw.loc[fam_file_raw["Gender"] == "F", "Gender"] = "2"
np.unique(fam_file_raw["Gender"])

array(['0', '1', '2'], dtype=object)

Add the family variables and the phenotype (not added for now)

In [169]:
fam_file_raw["FID"] = "combat" #ID for the whole study
fam_file_raw["IID_father"] = "0"
fam_file_raw["IID_mother"] = "0"
fam_file_raw["phenotype"] = -9 #this is no data for phenotype

Reorder

In [170]:
fam_file = fam_file_raw[["FID", "AGRF code", "IID_father", "IID_mother", "Gender", "phenotype"]]

In [171]:
fam_file

,FID,AGRF code,IID_father,IID_mother,Gender,phenotype
0,combat,8244TCOJ,0,0,1,-9
1,combat,6976MCJS,0,0,1,-9
2,combat,3484JSJJ,0,0,1,-9
3,combat,1389HSNM,0,0,2,-9
4,combat,7593DBAO,0,0,1,-9
...,...,...,...,...,...,...
1459,combat,2471AWMA,0,0,0,-9
1460,combat,6897TSSS,0,0,0,-9
1461,combat,6899DWOS,0,0,0,-9
1462,combat,6296TONJ,0,0,0,-9


Save without header:

In [172]:
fam_file.to_csv("data/plink_inputs_example/batch1_example.fam",
                sep="\t",
                header=None,
                index=False)

**Map file [Plink info](https://www.cog-genomics.org/plink/1.9/formats#map)**

A text file with no header line, and one line per variant with the following 3-4 fields:

- Chromosome code. PLINK 1.9 also permits contig names here, but most older programs do not.
- Variant identifier
- Position in morgans or centimorgans (optional; also safe to use dummy value of '0')
- Base-pair coordinate

We can get this information from the SNP map.

We could also get this information from the final report but, as shown in previous lines, we have only loaded the columns required for the lgen file because we have to load hundreds of final reports and concatenate them, so it will use less memory with less columns. The Indexes and positions are not duplicated in SNP map, only 1 row per SNP, being a much smaller file.

In [173]:
final_report1

,Sample Index,Sample ID,Sample Name,SNP Index,SNP Name,Chr,Position,GT Score,GC Score,Allele1 - AB,...,Y Raw,X,Y,B Allele Freq,Log R Ratio,SNP Aux,SNP,ILMN Strand,Top Genomic Sequence,Customer Strand
0,1,7684BSAO,NaN,1,1:103380393,1,102914837,0.7987,0.8136,A,...,958,0.324,0.360,0.4849,0.3097,0,[T/C],BOT,NaN,TOP
1,1,7684BSAO,NaN,2,1:109439680,1,108897058,0.8792,0.4803,A,...,131,0.779,0.019,0.0000,0.2660,0,[A/G],TOP,NaN,TOP
2,1,7684BSAO,NaN,3,1:110198788,1,109656166,0.8780,0.9154,A,...,198,1.170,0.028,0.0000,0.0312,0,[T/C],BOT,NaN,BOT
3,1,7684BSAO,NaN,4,1:110201112,1,109658490,0.8584,0.4545,A,...,728,1.154,0.072,0.0188,-0.0746,0,[G/C],BOT,NaN,TOP
4,1,7684BSAO,NaN,5,1:110201667,1,109659045,0.7815,0.7185,B,...,3269,0.120,1.291,0.9797,-0.0749,0,[T/C],BOT,NaN,BOT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654022,1,7684BSAO,NaN,654023,Y:8804321-A-C,Y,8936280,0.7229,0.4350,A,...,157,0.403,0.039,0.0000,-1.2565,0,[T/G],BOT,NaN,TOP
654023,1,7684BSAO,NaN,654024,Y:8852748-A-C,Y,8984707,0.8575,0.4533,A,...,147,0.730,0.028,0.0000,-0.3919,0,[T/G],BOT,NaN,TOP
654024,1,7684BSAO,NaN,654025,Y:8894070-A-T,Y,9026029,0.8175,0.4061,B,...,3099,0.017,1.436,1.0000,0.0729,0,[A/T],TOP,NaN,BOT
654025,1,7684BSAO,NaN,654026,Y:9145267-T-C,Y,9307658,0.7893,0.2966,A,...,201,0.334,0.059,0.0178,-1.3112,0,[T/C],BOT,NaN,BOT


Make a copy with copy(), so modifications to the data or indices of the copy will not be reflected in the original object

In [174]:
map_file_raw = snp_map.copy()
map_file_raw

,Index,Name,Chromosome,Position,GenTrain Score,SNP,ILMN Strand,Customer Strand,NormID
0,1,1:103380393,1,102914837,0.7987,[T/C],BOT,TOP,28
1,2,1:109439680,1,108897058,0.8792,[A/G],TOP,TOP,1
2,3,1:110198788,1,109656166,0.8780,[T/C],BOT,BOT,4
3,4,1:110201112,1,109658490,0.8584,[G/C],BOT,TOP,101
4,5,1:110201667,1,109659045,0.7815,[T/C],BOT,BOT,1
...,...,...,...,...,...,...,...,...,...
654022,654023,Y:8804321-A-C,Y,8936280,0.7229,[T/G],BOT,TOP,3
654023,654024,Y:8852748-A-C,Y,8984707,0.8575,[T/G],BOT,TOP,3
654024,654025,Y:8894070-A-T,Y,9026029,0.8175,[A/T],TOP,BOT,203
654025,654026,Y:9145267-T-C,Y,9307658,0.7893,[T/C],BOT,BOT,3


In [175]:
map_file_raw["centimorgans"] = 0 #dummy value of zero

In [176]:
map_file = map_file_raw[["Chromosome", "Name", "centimorgans", "Position"]]

In [177]:
map_file

,Chromosome,Name,centimorgans,Position
0,1,1:103380393,0,102914837
1,1,1:109439680,0,108897058
2,1,1:110198788,0,109656166
3,1,1:110201112,0,109658490
4,1,1:110201667,0,109659045
...,...,...,...,...
654022,Y,Y:8804321-A-C,0,8936280
654023,Y,Y:8852748-A-C,0,8984707
654024,Y,Y:8894070-A-T,0,9026029
654025,Y,Y:9145267-T-C,0,9307658


Save

In [178]:
map_file.to_csv("data/plink_inputs_example/batch1_example.map",
                sep="\t",
                header=None,
                index=False)

CHECK THE THREE FILES

Remove SNPs/samples not included in the lgen file!!! 

Change to ped file with PLINK 

In [181]:
import os 

os.system("ls data/plink_inputs_example")

batch1_example.fam
batch1_example.lgen
batch1_example.map


0

In [191]:
import os 

os.system("cd data/plink_inputs_example; plink --nonfounders --lfile batch1_example --recode --out batch1_example_plink")

#I have not checked this code

PLINK v1.90b6.27 64-bit (10 Dec 2022)          www.cog-genomics.org/plink/1.9/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to batch1_example_plink.log.
Options in effect:
  --lfile batch1_example
  --nonfounders
  --out batch1_example_plink
  --recode

64162 MB RAM detected; reserving 32081 MB for main workspace.


Error: Duplicate ID 'combat 1100JHJM'.


768

This error is caused because in the phenotype file, row 1423 is empty and then we have rows with sample ID but no phenotype, so we should remove all rows from 1423, 

In [ ]:
fam_file_raw.iloc[1422,:]

Full tutorial for plink

https://genomicsbootcamp.github.io/book/

Things to check when QC:

- Check the genome build. I have manually checked some SNPs and they have the position of hg38.
    - YOU ARE NOT ADDED CENTIMORGANS, SO BE CAREFUL WITH WHAT PLINK DO ABOUT LD.
- Remove zero chromosomes
    - also MT, X, Y and XY?
- Check strand, because the foward strand of illumina not always matches that of dbSNP
    - StrandScript is a software to check and solve that
- Check the illumina pdf report for the first batch (zip file in initial stuff), because they say there are three problematic samples.
    - these three sample also have unknown sex according to illumina data!!
- check sex between illumina and real sex data (pheno_data)
    - remove uknown sex individuals?
- check genotype calls that are I or D
    - how plink deals with this?
- ask David about the samples without phenotype in the excel file?

In [ ]:
%%bash

cd /home/dftortosa/Desktop

plink --file ILGSA24-17873 --freq --out ILGSA24-17873_analysis

In [ ]:
%%bash

cd data/example_data

#https://www.cog-genomics.org/plink/1.9/formats#lgen

#plink --lfile example_ILGSA24-17303_FinalReport1.txt --recode

#https://www.biostars.org/p/51928/

There are two batches (ILGSA24-17303 and ILGSA24-17873), being the data separated for these. 

- In ILGSA24-17303.zip, we have the final reports for each 216 samples, along with the sample and snp maps.
    - In the initial_stuff folder there is a zip called "ILGSA24-17303.zip" that I may downloaded from the initial location where this data was stored in summer 2022. There are Plink files, but I am not sure this is the correct data and I cannot find the final_report files.

- In 17873, we have the IDAT files with probs intensity from the microarrays used to genotype (first zips), the final reports (CAGRF20093767.zip) and a inputs for plink. But all of this only for 1248 individuals, not the whole cohort.
    - CAGRF20093767.zip includes the final reports of 1248 individuals, along with the sample and snp maps.

warning [CAGRF20093767.zip]:  32332459452 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [CAGRF20093767.zip]:  start of central directory not found;
  zipfile corrupt.
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)


CHECK WARNING

solution for the error: https://askubuntu.com/questions/54904/unzip-error-end-of-central-directory-signature-not-found

- Quality Control Procedures for Genome-Wide Association Studies
- A tutorial on conducting genome‐wide association studies: Quality control and statistical analysis
- Genetic prediction of complex traits with polygenic scores: a statistical review
- Addressing the challenges of polygenic scores in human genetic research